# Notes

This jupyter notebook is to check whether loading a different chunk for a snaphoot results in different masses.

This is because currently, plot_mass_fractions.ipynb results in the same values for every ingle chunk.

This notebook will load snapshot 67 chunk 0

It is coordination with mass_fraction_check_1.ipynb, mass_fraction_check_2.ipynb, and mass_fraction_check_4.ipynb
which will check snapshot 99 chunk 0, snapshot 99 chunk 3, and snapshot 67 chunk 3 respectively.

# Imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import yt
import trident
from trident import LightRay
import illustris_python as il
from astropy import units as u
from astropy import constants as c
from yt.utilities.cosmology import Cosmology
from matplotlib import pyplot as plt

/draco/u/cwalker/virtual_environments/yt-git/yt/utilities/logger.py:4: VisibleDeprecationWarning: The configuration file /u/cwalker/.config/yt/ytrc is deprecated in favor of /u/cwalker/.config/yt/yt.toml. Currently, both are present. Please manually remove the deprecated one to silence this warning.
Deprecated since v4.0.0 . This feature will be removed in v4.1.0
  from yt.config import ytcfg


# Choose snapshot and chunk to plot

In [2]:
snapshots=[67]
chunks=[0]

# Begin

In [3]:
######################################
#initialise arrays that hold all data#
######################################

#redshift array
allsnaps_redshifts=[]

#baryon arrays
#PartType0 (gas)
allsnaps_tot_PT0_arr = []
allsnaps_hal_PT0_arr = []
allsnaps_fil_PT0_arr = []
allsnaps_voi_PT0_arr = []
#PartType4 (stars)
allsnaps_tot_PT4_arr = []
allsnaps_hal_PT4_arr = []
allsnaps_fil_PT4_arr = []
allsnaps_voi_PT4_arr = []

#check arrays
allsnaps_dens_crit_array = []
allsnaps_m_p_array = []

################################
#loop over snapshots and chunks#
################################

for i in range(len(snapshots)):
    
    ######################################################
    #initialise arrays that hold individual snapshot data#
    ######################################################

    #redshift array
    redshifts=[]

    #baryon arrays
    #PartType0 (gas)
    tot_PT0_arr = []
    hal_PT0_arr = []
    fil_PT0_arr = []
    voi_PT0_arr = []
    #PartType4 (stars)
    tot_PT4_arr = []
    hal_PT4_arr = []
    fil_PT4_arr = []
    voi_PT4_arr = []

    #check arrays
    dens_crit_array = []
    m_p_array = []
    
    for j in range(len(chunks)):
    
        ###########################
        #select snapshot and chunk#
        ###########################

        snapshot = snapshots[i]
        chunk = chunks[j]
        print('processsing snapshot {0:02d}, chunk {1}'.format(snapshot,chunk))

        ######################
        #load snapshot header#
        ######################

        print('loading snapshot header')
        basepath = '/virgo/simulations/IllustrisTNG/TNG100-3/output/'
        header=il.groupcat.loadHeader(basepath,snapshot)
        print('header loaded.')

        ########################################################
        #extract redshift from header, append to redshift array#
        ########################################################

        redshifts.append(header['Redshift'])

        #####################
        #load Illustris data#
        #####################

        data='/virgo/simulations/IllustrisTNG/TNG100-3/output/snapdir_0{0:02d}/snap_0{0:02d}.{1}.hdf5'.format(snapshot,chunk)
        print('loading dataset: {0}'.format(data))
        ds=yt.load(data)
        print('data loaded.')

        ####################################
        #define proton mass in solar masses#
        ####################################

        print('calculating proton mass:')
        m_p = ds.quan((c.m_p).to(u.solMass).value,'Msun')
        print ('proton mass: {0}'.format(m_p))
        m_p_array.append(m_p)

        ds.index

        ############################################
        #calculate critical density of the Universe#
        ############################################

        print('Calculating critical density of the Universe')
        co=Cosmology()
        grav=ds.quan(6.6743e-11,'m**3/(kg*s**2)')
        H=co.hubble_parameter(0).in_units('km/s/Mpc')
        print('grav={0}'.format(grav))
        print('H0 = {0}'.format(H))
        dens_crit = ((3 * H**2)/(8*np.pi* grav))#.in_units('h**2/(code_length**3/(Msun*s**2))')
        print ('critical density is: {0}'.format(dens_crit))
        print('critical density in kg/m^3: {0}'.format(dens_crit.in_units('kg/m**3')))
        dens_crit_array.append(dens_crit)

        ad = ds.all_data()

        ##########################################################
        #convert dark matter density to units of critical density#
        ##########################################################

        #PartType0 (gas)
        dark_units_PT0 = ad['PartType0','SubfindDMDensity']*ds.quan(1e10,"(Msun/h)/((code_length)**3)")/dens_crit
        #PartType4 (stars)
        dark_units_PT4 = ad['PartType4','SubfindDMDensity']*ds.quan(1e10,"(Msun/h)/((code_length)**3)")/dens_crit

        ###############################################################
        #create Large-Scale Structure (LSS) masks for each matter type#
        ###############################################################

        #PartType0 (gas)
        voi_mask_PT0 = dark_units_PT0 < 0.1
        fil_mask_PT0 = np.logical_and(dark_units_PT0 >= 0.01, dark_units_PT0 < 57)
        hal_mask_PT0 = dark_units_PT0 >= 57 
        #PartType4 (stars)
        voi_mask_PT4 = dark_units_PT4 < 0.1
        fil_mask_PT4 = np.logical_and(dark_units_PT4 >= 0.01, dark_units_PT4 < 57)
        hal_mask_PT4 = dark_units_PT4 >= 57

        ##########################################################
        #calculate mass in each LSS for each baryonic matter type#
        ##########################################################

        #PartType0 (gas)
        tot_PT0 = np.sum(ad['PartType0','Masses'])
        hal_PT0 = np.sum(ad['PartType0','Masses'][hal_mask_PT0])
        fil_PT0 = np.sum(ad['PartType0','Masses'][fil_mask_PT0])
        voi_PT0 = np.sum(ad['PartType0','Masses'][voi_mask_PT0])
        #PartType4 (stars)
        tot_PT4 = np.sum(ad['PartType4','Masses'])
        hal_PT4 = np.sum(ad['PartType4','Masses'][hal_mask_PT4])
        fil_PT4 = np.sum(ad['PartType4','Masses'][fil_mask_PT4])
        voi_PT4 = np.sum(ad['PartType4','Masses'][voi_mask_PT4])

        ##########################
        #append results to arrays#
        ##########################

        #PartType0 (gas)
        tot_PT0_arr.append(tot_PT0)
        hal_PT0_arr.append(hal_PT0)
        fil_PT0_arr.append(fil_PT0) 
        voi_PT0_arr.append(voi_PT0) 
        #PartType4 (gas)
        tot_PT4_arr.append(tot_PT4)
        hal_PT4_arr.append(hal_PT4)
        fil_PT4_arr.append(fil_PT4) 
        voi_PT4_arr.append(voi_PT4)
        
    ######################################################    
    #append all results for a snapshot to the final array#
    ######################################################
    
    #PartType0 (gas)
    allsnaps_tot_PT0_arr.append(tot_PT0_arr)
    allsnaps_hal_PT0_arr.append(hal_PT0_arr)
    allsnaps_fil_PT0_arr.append(fil_PT0_arr) 
    allsnaps_voi_PT0_arr.append(voi_PT0_arr) 
    #PartType4 (gas)
    allsnaps_tot_PT4_arr.append(tot_PT4_arr)
    allsnaps_hal_PT4_arr.append(hal_PT4_arr)
    allsnaps_fil_PT4_arr.append(fil_PT4_arr) 
    allsnaps_voi_PT4_arr.append(voi_PT4_arr)    
    
    allsnaps_m_p_array.append(m_p_array)
    allsnaps_dens_crit_array.append(dens_crit_array)
    allsnaps_redshifts.append(redshifts)

    
    print('filled baryon arrays')
    
    

processsing snapshot 67, chunk 0
loading snapshot header
header loaded.
loading dataset: /virgo/simulations/IllustrisTNG/TNG100-3/output/snapdir_067/snap_067.0.hdf5


yt : [INFO     ] 2021-05-26 12:55:53,297 Calculating time from 6.653e-01 to be 2.710e+17 seconds
yt : [INFO     ] 2021-05-26 12:55:53,397 Parameters: current_time              = 2.7097180874458618e+17 s
yt : [INFO     ] 2021-05-26 12:55:53,398 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2021-05-26 12:55:53,398 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2021-05-26 12:55:53,399 Parameters: domain_right_edge         = [75000. 75000. 75000.]
yt : [INFO     ] 2021-05-26 12:55:53,400 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2021-05-26 12:55:53,401 Parameters: current_redshift          = 0.5030475232448832
yt : [INFO     ] 2021-05-26 12:55:53,401 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2021-05-26 12:55:53,402 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2021-05-26 12:55:53,402 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2021-05-26 12:55:53,402 Parameters: hubble_constant

data loaded.
calculating proton mass:
proton mass: 8.411856872862986e-58 Msun


yt : [INFO     ] 2021-05-26 12:55:58,080 Allocating for 3.743e+08 particles
Initializing refined index: 100%|█████████▉| 722/723 [23:38<00:01,  1.97s/it]  


Calculating critical density of the Universe
grav=6.6743e-11 m**3/(kg*s**2)
H0 = 71.0 km/(Mpc*s)
critical density is: 9.468720094206941e-27 kg/m**3
critical density in kg/m^3: 9.468720094206941e-27 kg/m**3
filled baryon arrays


In [4]:
#########################
#convert to numpy arrays#
#########################

#tot_PT0_arr=np.array(tot_PT0_arr)
#hal_PT0_arr=np.array(hal_PT0_arr)
#fil_PT0_arr=np.array(fil_PT0_arr)
#voi_PT0_arr=np.array(voi_PT0_arr)

#tot_PT4_arr=np.array(tot_PT4_arr)
#hal_PT4_arr=np.array(hal_PT4_arr)
#fil_PT4_arr=np.array(fil_PT4_arr)
#voi_PT4_arr=np.array(voi_PT4_arr)

allsnaps_tot_PT0_arr = np.array(allsnaps_tot_PT0_arr)
allsnaps_hal_PT0_arr = np.array(allsnaps_hal_PT0_arr)
allsnaps_fil_PT0_arr = np.array(allsnaps_fil_PT0_arr)
allsnaps_voi_PT0_arr = np.array(allsnaps_voi_PT0_arr)

allsnaps_tot_PT4_arr = np.array(allsnaps_tot_PT4_arr)
allsnaps_hal_PT4_arr = np.array(allsnaps_hal_PT4_arr)
allsnaps_fil_PT4_arr = np.array(allsnaps_fil_PT4_arr)
allsnaps_voi_PT4_arr = np.array(allsnaps_voi_PT4_arr)

allsnaps_redshifts = np.array(allsnaps_redshifts)
allsnaps_m_p_array = np.array(allsnaps_m_p_array)
allsnaps_dens_crit_array = np.array(allsnaps_dens_crit_array)

In [5]:
###############
#check lengths#
###############

print(len(snapshots))
print(allsnaps_redshifts.shape)
print(allsnaps_m_p_array.shape)
print(allsnaps_dens_crit_array.shape)
print(allsnaps_hal_PT0_arr.shape)
print(allsnaps_fil_PT0_arr.shape)
print(allsnaps_voi_PT0_arr.shape)
print(allsnaps_hal_PT4_arr.shape)
print(allsnaps_fil_PT4_arr.shape)
print(allsnaps_voi_PT4_arr.shape)


print(allsnaps_tot_PT0_arr.sum(axis=1).shape) #summing chunks for each snapshot
print(allsnaps_redshifts.mean(axis=1)) #recovering the redshift for each snapshot

1
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1,)
[0.50304752]


In [6]:
#print values
print(allsnaps_tot_PT0_arr)
print(allsnaps_hal_PT0_arr)
print(allsnaps_fil_PT0_arr)
print(allsnaps_voi_PT0_arr)

print(allsnaps_tot_PT4_arr)
print(allsnaps_hal_PT4_arr)
print(allsnaps_fil_PT4_arr)
print(allsnaps_voi_PT4_arr)

[[559385.39283825]]
[[116047.26667843]]
[[443338.11283495]]
[[16074.35808849]]
[[9059.99653625]]
[[9051.69578687]]
[[8.30074938]]
[[0.23881572]]
